# Analysis

Create a pandas DataFrame for saving metadata and analysis results

In [14]:
import itertools

In [15]:
from __future__ import print_function
import mdtraj as md
import numpy as np
import warnings
import MDAnalysis as mda
import itertools

/home/scottzhuang/anaconda3/envs/deepchem/lib/python3.5/site-packages/seaborn/apionly.py:9: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)
/home/scottzhuang/anaconda3/envs/deepchem/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/scottzhuang/anaconda3/envs/deepchem/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This 

In [18]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()

- TRAJ_NOTE format:

A.B_C_...Z

- A: strucuture name or PDB.
- B: mutation
- C: main ligand
- Z: interesting variables

e.g. 
- GABA_WT_PBT_POPC
- 4HFI_F238L_ethanol_pH46

In [24]:
traj_notes = ['URRO_pH3_dens_pH46_amber_largerbox', 'URRO_pH3_dens_pH70_amber_largerbox','URRO_pH5_dens_pH46_amber_largerbox','URRO_pH5_dens_pH70_amber_largerbox','URRO_pH7_dens_pH46_amber_largerbox','URRO_pH7_dens_pH70_amber_largerbox','4NPQ_pH70_amber_largerbox']
default_load_location = '/home/scottzhuang/eriklab/'
default_save_location = '/home/scottzhuang/pdc/'
default_skip = 10
default_rep = 4

In [25]:
traj_note_dic = {'traj_note': traj_notes, 
                 'load_location': ["".join(i) for i in itertools.product([default_load_location], traj_notes,['/multijob_rigid/'])],
                 'save_location':[default_save_location] * len(traj_notes), 
                 'skip':[default_skip] * len(traj_notes),
                 'rep': [4] * len(traj_notes)}

## Protein annnotations

In [8]:
subunit_dic = {0:0,1:1,2:2,3:3,4:4,5:0,-1:4}

In [9]:
subunit_dic_mda = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'A'}

In [11]:
subunit_type = {0:'WT',1:'WT',2:'WT',3:'WT',4:'WT',5:'WT'}

In [12]:
pore_annotation = {'-2': '(resid 222 and resname GLU)',
                   '2': '(resid 226 and resname THR)',
                   '6': '(resid 230 and resname SER)',
                   '9':  '(resid 233 and resname ILE)',
                   '13': '(resid 237 and resname ALA)',
                   '16': '(resid 240 and resname ILE)',
                   '20': '(resid 244 and resname THR)'}

In [13]:
secondary_structure_annotation = {
    'WT':{
        'ECD': '(resSeq 10 to 192)',

        'TMD': '(resSeq 193 to 315)'
    }
    }

In [14]:
domain_structure_annotation = {
    'WT':{
        'M1': '(resSeq 193 to 218)',
        'M2': '(resSeq 220 to 244)',
        'M3': '(resSeq 256 to 283)',
        'M4': '(resSeq 285 to 315)',
        'loop_C': '(resSeq 162 to 190)',
        'M2_M3_loop': '(resSeq 244 to 256)',
    }
}

## Create Dataframe and metadata

In [30]:
def create_md_dataframe():    
    return pd.DataFrame(columns=list(['MD_name', 'replicate', 'frame', 'traj_time', 'system', 'id', 'ligand', 'note']))

In [31]:
md_data = create_md_dataframe()

In [32]:
def append_metadata(traj_note, rep, ident, system, location, skip = default_skip):
        rep_data = []
        traj_note_split = traj_note.split('_')
        try:
            top_location = traj_note + '/rep' + rep + '/' + "md.ca.gro"
            traj_location = traj_note+ '/rep' + rep + '/' + 'md' + ".skip" + str(skip) + ".ca.xtc"
            traj = md.load(location + traj_location,top= location + top_location)          
            md_name = traj_note_split[0]
            ligand = traj_note_split[1:-1] ##in another case
            timestep = 5 ##in this case
            note = traj_note.split('/')[-1]
            for i in range(0,traj.n_frames):
                rep_data.append([md_name, rep, i,  i * 0.2 * timestep * skip, system, ident,ligand,note])
        except:
            print(traj_note + ' not found.')
        return rep_data
    
    
meta_data = Parallel(n_jobs=num_cores)(delayed(append_metadata)(traj_note = traj_note_dic['traj_note'][i//4], 
                                                        rep = str(i%4 + 1),
                                                        ident = i,
                                                        system = i//4,
                                                        location = traj_note_dic['save_location'][i//4],
                                                       skip = traj_note_dic['skip'][i//4], 
                                               )
                           for i in range(0,len(traj_note_dic['traj_note']) * 4))
for i in range(0,len(traj_note_dic['traj_note']) * 4):
    md_data = md_data.append(pd.DataFrame(meta_data[i],columns=list(['MD_name','replicate','frame','traj_time','system','id','ligand','note'])),ignore_index=True)
md_data['frame'] =md_data['frame'].apply(int)
md_data['traj_time'] =md_data['traj_time'].apply(float)
md_data['replicate'] =md_data['replicate'].apply(int)
md_data['system'] = md_data['system'].apply(int)

In [33]:
md_data.head()

,MD_name,replicate,frame,traj_time,system,id,ligand,note
0,URRO,1,0,0.0,0,0,"[pH3, dens, pH46, amber]",URRO_pH3_dens_pH46_amber_largerbox
1,URRO,1,1,10.0,0,0,"[pH3, dens, pH46, amber]",URRO_pH3_dens_pH46_amber_largerbox
2,URRO,1,2,20.0,0,0,"[pH3, dens, pH46, amber]",URRO_pH3_dens_pH46_amber_largerbox
3,URRO,1,3,30.0,0,0,"[pH3, dens, pH46, amber]",URRO_pH3_dens_pH46_amber_largerbox
4,URRO,1,4,40.0,0,0,"[pH3, dens, pH46, amber]",URRO_pH3_dens_pH46_amber_largerbox


## RMSD

In [34]:
def append_rmsd_data(traj_note, rep, location, skip = default_skip):
    try:
#        print(traj_note, rep, subunit, location, skip)
        top_location = traj_note + '/rep' + rep + '/' + "md.ca.pdb"
        traj_location = traj_note+ '/rep' + rep + '/' + 'md' + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top= location + top_location)
        rmsd = []
        rmsd.append((md.rmsd(traj, traj)*10))
        return (np.asarray(rmsd).transpose().ravel())
    except:
        print(traj_note + ' not found.')

rmsd_data = Parallel(n_jobs=num_cores)(delayed(append_rmsd_data)(traj_note = traj_note_dic['traj_note'][i//4], 
                                                        rep = str(i%4 + 1),
                                                        location = traj_note_dic['save_location'][i//4],
                                                       skip = traj_note_dic['skip'][i//4], 
                                               )
                           for i in range(0,len(traj_note_dic['traj_note']) * 4))

md_data['rmsd'] = [x for x in np.hstack(rmsd_data) if x is not None]

## Pore Hydration

In [ ]:
def append_resid_hydration_data(traj_note,rep,resid_selection,location, skip=default_skip):
    try:
        hydration_data = []
        top_location = traj_note + '/rep' + rep + '/' + 'md' + ".system.pdb"
        traj_location = traj_note+ '/rep' + rep + '/' + 'md' + ".skip" + str(skip) + ".system.xtc"
        traj = mda.Universe(location + top_location,location + traj_location)
        hydration = traj.select_atoms("(cyzone 9 2 -2 (" + resid_selection + "))  and resname SOL and name OW",updating = True)
        for i in range(0,traj.trajectory.n_frames):
            traj.trajectory[i]
            hydration_data.append(hydration.n_atoms)
        return hydration_data
    except:
        print(traj_note + ' not found.')
for resid in pore_annotation.keys():
    resid_hydration_data = Parallel(n_jobs=num_cores)(delayed(append_resid_hydration_data)(traj_note = traj_note_dic['traj_note'][i//4], 
                                                            rep = str(i%4 + 1),
                                                            resid_selection = pore_annotation[resid],
                                                            location = traj_note_dic['save_location'][i//4],
                                                           skip = traj_note_dic['skip'][i//4], 
                                                   )
                               for i in range(0,len(traj_note_dic['traj_note']) * 4))

    md_data['pore_hydration_resid_' + resid] = [x for x in np.hstack(resid_hydration_data) if x is not None]

etc.

## Save results

In [54]:
md_data.to_pickle('GLIC_analysis_results.pickle')